# 🎮 Trust Dynamics Playground

*One-click trust evolution simulator - explore how trust builds and breaks!*

This notebook implements game-theoretic trust dynamics based on:
- Axelrod's evolution of cooperation principles
- Nowak's benefit-to-cost ratios for cooperation
- Empirically validated forgiveness factors (0.1-0.5 range)

**Note**: Trust decay and cooperation thresholds are configurable parameters, not universal constants.

In [ ]:
# Install required packages (runs automatically in Colab)
!pip install plotly pandas numpy ipywidgets -q

In [ ]:
#@title 🎮 Trust Dynamics Playground { display-mode: "form" }
#@markdown One-click trust evolution simulator - explore how trust builds and breaks!

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, Button, Output
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns

# Beautiful color palette
TRUST_COLORS = {
    'high_trust': '#4ECDC4',      # Teal - stable trust
    'building': '#F7B801',        # Gold - emerging trust  
    'neutral': '#95A99C',         # Gray - uncertain
    'betrayal': '#FF6B6B',        # Red - broken trust
    'background': '#F8F9FA'       # Light gray
}

class TrustGame:
    """Core trust game mechanics based on established game theory"""
    def __init__(self, trust_decay=0.1, forgiveness_rate=0.2, error_rate=0.05):
        """
        Initialize with configurable parameters.
        
        Args:
            trust_decay: Rate of trust reduction over time (0-1)
            forgiveness_rate: How quickly trust recovers (0-1)
            error_rate: Probability of mistakes in actions (0-1)
        """
        self.trust_decay = trust_decay
        self.forgiveness_rate = forgiveness_rate
        self.error_rate = error_rate
        self.history = []
        
        # Nowak's benefit-to-cost ratio for cooperation stability
        self.benefit_cost_ratio = 3.0  # b/c ratio, cooperation stable when > 1
        
    def apply_error(self, intended_action):
        """Simulate mistakes in execution (empirically ~10% error rate disrupts cooperation)"""
        if np.random.random() < self.error_rate:
            return 'defect' if intended_action == 'cooperate' else 'cooperate'
        return intended_action
        
    def play_round(self, p1_action, p2_action, current_trust):
        """Calculate payoffs and trust updates based on game theory"""
        # Apply execution errors
        actual_p1 = self.apply_error(p1_action)
        actual_p2 = self.apply_error(p2_action)
        
        # Payoff matrix with benefit-to-cost considerations
        if actual_p1 == 'cooperate' and actual_p2 == 'cooperate':
            payoffs = (3, 3)  # Mutual benefit
            trust_change = 0.1
        elif actual_p1 == 'cooperate' and actual_p2 == 'defect':
            payoffs = (0, 5)  # Exploited
            trust_change = -0.3
        elif actual_p1 == 'defect' and actual_p2 == 'cooperate':
            payoffs = (5, 0)  # Exploiter
            trust_change = -0.3
        else:  # both defect
            payoffs = (1, 1)  # Mutual defection
            trust_change = -0.05
            
        # Update trust with decay and bounds
        new_trust = current_trust + trust_change
        new_trust = new_trust * (1 - self.trust_decay) + self.forgiveness_rate * (0.5 - new_trust)
        new_trust = max(0, min(1, new_trust))
        
        self.history.append({
            'round': len(self.history) + 1,
            'p1_intended': p1_action,
            'p2_intended': p2_action,
            'p1_actual': actual_p1,
            'p2_actual': actual_p2,
            'p1_payoff': payoffs[0],
            'p2_payoff': payoffs[1],
            'trust_level': new_trust,
            'trust_change': trust_change,
            'error_occurred': actual_p1 != p1_action or actual_p2 != p2_action
        })
        
        return payoffs, new_trust

class TrustVisualizer:
    """Beautiful trust visualizations"""
    def __init__(self):
        self.setup_style()
        
    def setup_style(self):
        """Clean, minimal aesthetic"""
        plt.style.use('seaborn-v0_8-whitegrid')
        plt.rcParams['figure.facecolor'] = TRUST_COLORS['background']
        plt.rcParams['axes.facecolor'] = 'white'
        
    def plot_trust_evolution(self, history_df):
        """Interactive trust timeline"""
        fig = go.Figure()
        
        # Trust line
        fig.add_trace(go.Scatter(
            x=history_df['round'],
            y=history_df['trust_level'],
            mode='lines+markers',
            name='Trust Level',
            line=dict(color=TRUST_COLORS['building'], width=3),
            marker=dict(size=8),
            hovertemplate='Round %{x}<br>Trust: %{y:.2f}<extra></extra>'
        ))
        
        # Color regions by trust level
        for idx, row in history_df.iterrows():
            if row['trust_level'] > 0.7:
                color = TRUST_COLORS['high_trust']
            elif row['trust_level'] > 0.3:
                color = TRUST_COLORS['building']
            else:
                color = TRUST_COLORS['betrayal']
                
            fig.add_shape(
                type="rect",
                x0=row['round']-0.5, x1=row['round']+0.5,
                y0=0, y1=row['trust_level'],
                fillcolor=color,
                opacity=0.3,
                line_width=0
            )
        
        # Betrayal markers
        betrayals = history_df[history_df['trust_change'] < -0.2]
        fig.add_trace(go.Scatter(
            x=betrayals['round'],
            y=betrayals['trust_level'],
            mode='markers',
            name='Betrayal',
            marker=dict(
                size=15,
                color=TRUST_COLORS['betrayal'],
                symbol='x'
            ),
            showlegend=True
        ))
        
        # Error markers
        errors = history_df[history_df['error_occurred']]
        if len(errors) > 0:
            fig.add_trace(go.Scatter(
                x=errors['round'],
                y=errors['trust_level'],
                mode='markers',
                name='Error',
                marker=dict(
                    size=10,
                    color='orange',
                    symbol='diamond'
                ),
                showlegend=True
            ))
        
        fig.update_layout(
            title="Trust Evolution Over Time",
            xaxis_title="Round",
            yaxis_title="Trust Level",
            yaxis_range=[0, 1],
            hovermode='x unified',
            plot_bgcolor='white',
            height=500
        )
        
        return fig
    
    def plot_strategy_outcomes(self, history_df):
        """Strategy effectiveness visualization"""
        # Calculate cumulative payoffs
        history_df['p1_cumulative'] = history_df['p1_payoff'].cumsum()
        history_df['p2_cumulative'] = history_df['p2_payoff'].cumsum()
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=history_df['round'],
            y=history_df['p1_cumulative'],
            mode='lines',
            name='Player 1',
            line=dict(color='#4ECDC4', width=3)
        ))
        
        fig.add_trace(go.Scatter(
            x=history_df['round'],
            y=history_df['p2_cumulative'],
            mode='lines',
            name='Player 2',
            line=dict(color='#F7B801', width=3)
        ))
        
        fig.update_layout(
            title="Cumulative Payoffs",
            xaxis_title="Round",
            yaxis_title="Total Score",
            hovermode='x unified',
            plot_bgcolor='white',
            height=400
        )
        
        return fig

# Initialize game components
game = TrustGame()
visualizer = TrustVisualizer()

# Interactive simulation
output = Output()

@interact(
    strategy_p1=Dropdown(
        options=['Always Cooperate', 'Always Defect', 'Tit for Tat', 'Generous Tit for Tat', 'Random'],
        value='Tit for Tat',
        description='Player 1:'
    ),
    strategy_p2=Dropdown(
        options=['Always Cooperate', 'Always Defect', 'Tit for Tat', 'Generous Tit for Tat', 'Random'],
        value='Tit for Tat',
        description='Player 2:'
    ),
    initial_trust=FloatSlider(
        min=0, max=1, step=0.1, value=0.5,
        description='Initial Trust:'
    ),
    num_rounds=IntSlider(
        min=10, max=100, step=10, value=50,
        description='Rounds:'
    ),
    trust_decay=FloatSlider(
        min=0, max=0.3, step=0.01, value=0.05,
        description='Trust Decay:'
    ),
    error_rate=FloatSlider(
        min=0, max=0.2, step=0.01, value=0.05,
        description='Error Rate:'
    )
)
def run_trust_simulation(strategy_p1, strategy_p2, initial_trust, num_rounds, trust_decay, error_rate):
    with output:
        clear_output(wait=True)
        
        # Reset game with new parameters
        global game
        game = TrustGame(trust_decay=trust_decay, error_rate=error_rate)
        current_trust = initial_trust
        last_p2_action = 'cooperate'
        
        # Strategy functions
        def get_action(strategy, opponent_last_action, trust_level):
            if strategy == 'Always Cooperate':
                return 'cooperate'
            elif strategy == 'Always Defect':
                return 'defect'
            elif strategy == 'Tit for Tat':
                return opponent_last_action
            elif strategy == 'Generous Tit for Tat':
                # Forgive defection with 30% probability (empirically validated)
                if opponent_last_action == 'defect' and np.random.random() < 0.3:
                    return 'cooperate'
                return opponent_last_action
            else:  # Random
                return 'cooperate' if np.random.random() < trust_level else 'defect'
        
        # Run simulation
        for round in range(num_rounds):
            p1_action = get_action(strategy_p1, last_p2_action, current_trust)
            p2_action = get_action(strategy_p2, p1_action, current_trust)
            
            payoffs, current_trust = game.play_round(p1_action, p2_action, current_trust)
            last_p2_action = game.history[-1]['p2_actual']  # Use actual action (may differ due to errors)
        
        # Visualize results
        history_df = pd.DataFrame(game.history)
        
        # Trust evolution
        trust_fig = visualizer.plot_trust_evolution(history_df)
        trust_fig.show()
        
        # Strategy outcomes
        payoff_fig = visualizer.plot_strategy_outcomes(history_df)
        payoff_fig.show()
        
        # Summary statistics
        print("\n📊 Summary Statistics:")
        print(f"Final Trust Level: {current_trust:.2f}")
        print(f"Average Trust: {history_df['trust_level'].mean():.2f}")
        print(f"Trust Volatility: {history_df['trust_level'].std():.2f}")
        print(f"\nPlayer 1 ({strategy_p1}):")
        print(f"  Total Score: {history_df['p1_payoff'].sum()}")
        print(f"  Cooperation Rate: {(history_df['p1_intended'] == 'cooperate').mean():.1%}")
        print(f"\nPlayer 2 ({strategy_p2}):")
        print(f"  Total Score: {history_df['p2_payoff'].sum()}")
        print(f"  Cooperation Rate: {(history_df['p2_intended'] == 'cooperate').mean():.1%}")
        
        # Error analysis
        error_count = history_df['error_occurred'].sum()
        if error_count > 0:
            print(f"\n⚠️ Execution Errors: {error_count} rounds ({error_count/num_rounds:.1%})")
        
        # Cooperation stability analysis
        cooperation_rounds = ((history_df['p1_actual'] == 'cooperate') & 
                            (history_df['p2_actual'] == 'cooperate')).sum()
        print(f"\n🤝 Mutual Cooperation: {cooperation_rounds} rounds ({cooperation_rounds/num_rounds:.1%})")
        
        # Scientific note
        print("\n📌 Note: These parameters are configurable, not universal constants.")
        print("   Optimal values depend on your specific context and goals.")

display(output)

#@markdown ---
#@markdown ### 🎯 Try These Experiments:
#@markdown 1. **Error Tolerance**: How do different error rates affect cooperation?
#@markdown 2. **Decay Dynamics**: Find the trust decay threshold for your strategies
#@markdown 3. **Forgiveness Benefits**: Compare Tit-for-Tat vs Generous Tit-for-Tat
#@markdown 4. **Trust Recovery**: Start with low trust and try to build it up

## 📚 Mathematical Foundations

This implementation is based on:

### Nowak's Cooperation Conditions
- Direct reciprocity requires: `b/c > 1/(1-w)` where `w` is probability of future interaction
- Network reciprocity requires: `b/c > k` where `k` depends on network structure

### Empirically Validated Parameters
- Error rates above ~10% can disrupt cooperation
- Forgiveness probabilities of 0.1-0.5 enable recovery from error cycles
- Trust decay and recovery are asymmetric processes

### Key Insights
- No universal thresholds exist - parameters are context-dependent
- Generous strategies often outperform strict reciprocity
- Small amounts of noise can actually stabilize cooperation